In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import time  # For tracking time
import winsound 
import pyttsx3# For beep sound

# Load the pre-trained model for eye and yawning detection
model = load_model('driver_drowsiness.h5')  # Replace with your actual model path

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Function to extract the eye region using landmarks
def extract_eye_region(frame, landmarks, eye_indices, margin=5):
    eye_points = []
    for index in eye_indices:
        eye_points.append((int(landmarks[index].x * frame.shape[1]), int(landmarks[index].y * frame.shape[0])))

    x_coords = [point[0] for point in eye_points]
    y_coords = [point[1] for point in eye_points]

    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)

    x_min = max(x_min - margin, 0)
    y_min = max(y_min - margin, 0)
    x_max = min(x_max + margin, frame.shape[1])
    y_max = min(y_max + margin, frame.shape[0])

    eye_region = frame[y_min:y_max, x_min:x_max]
    return eye_region

# Preprocess for eyes open/closed detection (using MediaPipe landmarks)
def preprocess_eye_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
        results = face_mesh.process(frame_rgb)
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                left_eye_indices = [33, 160, 158, 133, 153, 144]
                right_eye_indices = [362, 385, 387, 263, 373, 380]

                left_eye = extract_eye_region(frame_bgr, face_landmarks.landmark, left_eye_indices, margin=10)
                right_eye = extract_eye_region(frame_bgr, face_landmarks.landmark, right_eye_indices, margin=10)

                left_eye_resized = None
                right_eye_resized = None

                if left_eye is not None:
                    left_eye_resized = cv2.resize(left_eye, (150, 150))
                if right_eye is not None:
                    right_eye_resized = cv2.resize(right_eye, (150, 150))

                return left_eye_resized, right_eye_resized
        return None, None

# Function to play a beep sound
def play_beep():
    frequency = 1000  # Set frequency to 1000 Hz
    duration = 1000   # Set duration to 1000 ms (1 second)
    winsound.Beep(frequency, duration)

def preprocess_image_for_yawn(frame):
    """
    Detects faces and preprocesses the cropped face image for yawning detection.
    Returns preprocessed image and face coordinates.
    """
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_classifier.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]  # Crop the face region
        resized_face = cv2.resize(face, (150, 150))  # Resize to 150x150 (model input size)
        
        if len(resized_face.shape) == 2:  # Grayscale image
            resized_face = cv2.cvtColor(resized_face, cv2.COLOR_GRAY2RGB)  # Convert to 3 channels
        
        normalized_face = resized_face / 255.0  # Normalize pixel values to [0, 1]
        return np.expand_dims(normalized_face, axis=0), (x, y, w, h)  # Return face and coordinates
    
    return None, None  # No face detected
def text_to_speech(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)  # Speed (words per minute)
    engine.setProperty('volume', 1.0)  # Volume (0.0 to 1.0)
    engine.say(text)
    engine.runAndWait()

# Initialize counters and flags
yawn_count = 0
yawn_last_detected_time = 0  # Track last yawn detection time
yawn_detection_cooldown = 2  # Cooldown period in seconds

drowsiness_start_time = None
drowsiness_alert_triggered = False
DROWSINESS_ALERT_TIMEOUT = 10  # seconds

# Open webcam for real-time detection
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame for eye detection
    left_eye_resized, right_eye_resized = preprocess_eye_frame(frame)

    # Handle eye drowsiness detection
    if left_eye_resized is not None:
        processed_eye_frame = np.expand_dims(left_eye_resized, axis=0) / 255.0
        prediction_probabilities = model.predict(processed_eye_frame)

        labels = ['Closed', 'Open', 'no_yawn', 'yawn']
        predicted_label = labels[np.argmax(prediction_probabilities[0])]

        if predicted_label == "Closed":
            if drowsiness_start_time is None:
                drowsiness_start_time = time.time()
            elif time.time() - drowsiness_start_time >= 3:  # Check if eyes are closed for 3 seconds
                drowsiness_detected = True
        else:
            drowsiness_start_time = None
            drowsiness_detected = False

        # Display the prediction for eyes
        cv2.putText(frame, f"Prediction: {predicted_label}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Check if drowsiness alert is triggered
    if drowsiness_detected and not drowsiness_alert_triggered:
        cv2.putText(frame, "DROWSINESS DETECTED", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        winsound.Beep(2000, 1000)

    # Process frame for yawning detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale for face detection
    processed_frame, face_coords = preprocess_image_for_yawn(gray_frame)

    if processed_frame is not None:
        # Predict yawning probability
        predictions = model.predict(processed_frame)
        yawn_probability = predictions[0][3]  # Probability for the "Yawn" class
        
        # Visualize detection results
        if yawn_probability > 0.8 and time.time() - yawn_last_detected_time >= yawn_detection_cooldown:  
            # Only detect a new yawn if cooldown period has passed
            cv2.putText(frame, "Yawning Detected!", (300, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.rectangle(frame, (face_coords[0], face_coords[1]), 
                          (face_coords[0] + face_coords[2], face_coords[1] + face_coords[3]), 
                          (0, 0, 255), 2)
            yawn_count += 1  # Increment yawning count
            yawn_last_detected_time = time.time()  # Update the last detected time
        else:
            cv2.putText(frame, "No Yawning", (300, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Display the yawning count on the screen
        cv2.putText(frame, f"Yawns Detected: {yawn_count}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        # Check if yawn count exceeds 5
        if yawn_count > 5:
            text_to_speech("You yawned multiple time please take a break")  # Play Beep sound alert
            yawn_count = 0  # Reset yawning count after the alert
    else:
        cv2.putText(frame, "Face Not Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Display the video feed
    cv2.imshow("Yawning and Drowsiness Detection", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 